# 課題3(データ補完の効果)

**[課題1]** 次のコードをコピーして実行し、結果も含めたノートブックを作成して提出しなさい

**[課題2]** pandasのシンプルな補完を使って動作を確認してみなさい

In [ ]:
#メルボルンの住宅データを取り出し、pandas形式で表示する。
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from sklearn import preprocessing
pd.set_option("display.max_rows", 101)
pd.set_option("display.max_columns", 101)
#使用カラムを限定する
usecols=['Rooms', 'Type', 'Price', 'Method', 'SellerG',
        'Distance', 'Bedroom2', 'Bathroom', 'Car',
       'Landsize', 'BuildingArea', 'YearBuilt', 'Propertycount']
if not os.path.exists('Melbourne_housing_FULL.csv'):
    !wget "https://drive.google.com/uc?export=download&id=1Ow53Wl7g40Pr1ExUoJ4P-IrSejpzUalt" -O Melbourne_housing_FULL.csv #重複してダウンロードすることを避ける

df=pd.read_csv("Melbourne_housing_FULL.csv",usecols=usecols)

df #pandas形式ではprintは省略してよい

,Rooms,Type,Price,Method,SellerG,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Propertycount
0,2,h,NaN,SS,Jellis,2.5,2.0,1.0,1.0,126.0,NaN,NaN,4019.0
1,2,h,1480000.0,S,Biggin,2.5,2.0,1.0,1.0,202.0,NaN,NaN,4019.0
2,2,h,1035000.0,S,Biggin,2.5,2.0,1.0,0.0,156.0,79.0,1900.0,4019.0
3,3,u,NaN,VB,Rounds,2.5,3.0,2.0,1.0,0.0,NaN,NaN,4019.0
4,3,h,1465000.0,SP,Biggin,2.5,3.0,2.0,0.0,134.0,150.0,1900.0,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34852,4,h,1480000.0,PI,Jas,6.3,4.0,1.0,3.0,593.0,NaN,NaN,6543.0
34853,2,h,888000.0,SP,Sweeney,6.3,2.0,2.0,1.0,98.0,104.0,2018.0,6543.0
34854,2,t,705000.0,S,Jas,6.3,2.0,1.0,2.0,220.0,120.0,2000.0,6543.0
34855,3,h,1140000.0,SP,hockingstuart,6.3,NaN,NaN,NaN,NaN,NaN,NaN,6543.0


このデータを用いて家賃以外の情報から家賃を当てることを考える。なお、このデータにはpriceの情報が抜けているものがあるので初めにこれを削除する。

In [ ]:
df.dropna(subset=['Price'])
#df[df["Price"]==df["Price"]] というコードでも同様に実現できる。
#nan同士の比較では同じFALSEを返すという仕組みを利用している。

,Rooms,Type,Price,Method,SellerG,Distance,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Propertycount
1,2,h,1480000.0,S,Biggin,2.5,2.0,1.0,1.0,202.0,NaN,NaN,4019.0
2,2,h,1035000.0,S,Biggin,2.5,2.0,1.0,0.0,156.0,79.0,1900.0,4019.0
4,3,h,1465000.0,SP,Biggin,2.5,3.0,2.0,0.0,134.0,150.0,1900.0,4019.0
5,3,h,850000.0,PI,Biggin,2.5,3.0,2.0,1.0,94.0,NaN,NaN,4019.0
6,4,h,1600000.0,VB,Nelson,2.5,3.0,1.0,2.0,120.0,142.0,2014.0,4019.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34852,4,h,1480000.0,PI,Jas,6.3,4.0,1.0,3.0,593.0,NaN,NaN,6543.0
34853,2,h,888000.0,SP,Sweeney,6.3,2.0,2.0,1.0,98.0,104.0,2018.0,6543.0
34854,2,t,705000.0,S,Jas,6.3,2.0,1.0,2.0,220.0,120.0,2000.0,6543.0
34855,3,h,1140000.0,SP,hockingstuart,6.3,NaN,NaN,NaN,NaN,NaN,NaN,6543.0


In [ ]:
df.dropna(subset=['Price'], inplace=True) #inplaceでdfそのものを修正する
print("全データ数")
print(len(df["Rooms"])) # Roomsに欠損がないことを知っているので
print("\n欠損数の確認")
print(df.isnull().sum())
#カテゴリ値を数値変換
le = preprocessing.LabelEncoder()
df[[ 'Type', 'Method', 'SellerG']]=df[[ 'Type', 'Method', 'SellerG']].apply(le.fit_transform)
df_train=df.drop("Price",axis=1)
df_label=df["Price"].values
print("\n訓練データの確認")
print(df_train)
print("\n訓練ラベルの確認(つまりお部屋のお値段、これを当てる)")
print(df_label)

全データ数
27247

欠損数の確認
Rooms                0
Type                 0
Price                0
Method               0
SellerG              0
Distance             1
Bedroom2          6441
Bathroom          6447
Car               6824
Landsize          9265
BuildingArea     16591
YearBuilt        15163
Propertycount        3
dtype: int64

訓練データの確認
       Rooms  Type  Method  SellerG  Distance  Bedroom2  Bathroom  Car  \
1          2     0       1       32       2.5       2.0       1.0  1.0   
2          2     0       1       32       2.5       2.0       1.0  0.0   
4          3     0       3       32       2.5       3.0       2.0  0.0   
5          3     0       0       32       2.5       3.0       2.0  1.0   
6          4     0       4      206       2.5       3.0       1.0  2.0   
...      ...   ...     ...      ...       ...       ...       ...  ...   
34852      4     0       0      140       6.3       4.0       1.0  3.0   
34853      2     0       3      290       6.3       2.0       2.0 

In [ ]:
import lightgbm as lgb
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

def run_model(df_train,df_label):
  params={
        'objective': 'regression',
        'random_state' : 1,
        "metric": "rmse",
        } 
#今回は回帰解析であるのでregressionを指定する。random_stateは1。これは同じ結果が出るように.metricはrmseが小さくなるように推定していく。

  kfold = 5 #5個のグループにデータを分割する。
  score = 0

  kf = KFold(n_splits=kfold,shuffle=True,random_state=94) #交差検証する関数を定義。5個のブロックにシャッフルで分割
  importance = 0 #どのデータが最も答えに寄与しているかをの順序を出すための準備
  for i,(train_index,test_index) in enumerate(kf.split(df_train,df_label)): #kf.splitでtrainとlabelのデータを5個に分割.これをtrain_indexとtest_indexに埋めていく
        print('[Fold %d/%d]' % (i + 1, kfold)) #Foldのデータ1,2,3,4,5
        X_train, X_valid = df_train.iloc[train_index], df_train.iloc[test_index] #train_indexなどは「選択した要素の配列」0から要素数あり、kfoldの確率で抜けている
        y_train, y_valid = df_label[train_index], df_label[test_index] #上と同様だが、要素が一つしかないのでこれでよい
        dtrain = lgb.Dataset(X_train, label=y_train) #まとめてデータセットを構成する
        dvalid = lgb.Dataset(X_valid, label=y_valid)
        bst = lgb.train(params, dtrain, num_boost_round=1000,valid_sets=[dtrain, dvalid],early_stopping_rounds=50,verbose_eval=100)
        # LightGBMで学習、1000回試行して、ベストなものを返す
        importance += pd.DataFrame(bst.feature_importance(), index=df_train.columns, columns=['importance'])
        score+=bst.best_score["valid_1"]['rmse']
  return importance,score/5

初めに補完せずに欠損を含むデータを利用して学習させる。

In [ ]:
imp,score = run_model(df_train,df_label)
print(score)
print(imp)
scored = score

[Fold 1/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 263065	valid_1's rmse: 361421
[200]	training's rmse: 234052	valid_1's rmse: 352385
[300]	training's rmse: 216683	valid_1's rmse: 349947
[400]	training's rmse: 204275	valid_1's rmse: 349206
Early stopping, best iteration is:
[380]	training's rmse: 206508	valid_1's rmse: 348965
[Fold 2/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 271639	valid_1's rmse: 320021
[200]	training's rmse: 242773	valid_1's rmse: 311486
[300]	training's rmse: 225266	valid_1's rmse: 308975
[400]	training's rmse: 212653	valid_1's rmse: 308360
Early stopping, best iteration is:
[353]	training's rmse: 218298	valid_1's rmse: 308194
[Fold 3/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 275512	valid_1's rmse: 301153
[200]	training's rmse: 245402	valid_1's rmse: 292408
[300]	training's rmse: 227787	valid_1's rmse: 288409
[400]	training's rmse:

In [ ]:
#欠損値を補完関数IterativeImputerのデフォルトのBaysian Ridgeで補完
imp = IterativeImputer(max_iter=50, random_state=5) # 方法の定義
df_train_fi = pd.DataFrame(imp.fit_transform(df_train)) # 定義した方法を用いて実際に補完
df_train_fi.columns = df_train.columns # カラム情報もコピーしておく
imp,score = run_model(df_train_fi,df_label)
print(score)
scorei = score

[Fold 1/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 262844	valid_1's rmse: 363447
[200]	training's rmse: 231079	valid_1's rmse: 356238
[300]	training's rmse: 210905	valid_1's rmse: 353138
Early stopping, best iteration is:
[347]	training's rmse: 203492	valid_1's rmse: 352750
[Fold 2/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 273414	valid_1's rmse: 329334
[200]	training's rmse: 241484	valid_1's rmse: 319327
[300]	training's rmse: 220559	valid_1's rmse: 316351
[400]	training's rmse: 205133	valid_1's rmse: 315500
[500]	training's rmse: 192561	valid_1's rmse: 314929
[600]	training's rmse: 181615	valid_1's rmse: 314875
Early stopping, best iteration is:
[556]	training's rmse: 186354	valid_1's rmse: 314720
[Fold 3/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 277224	valid_1's rmse: 299220
[200]	training's rmse: 244078	valid_1's rmse: 291363
[300]	training's rmse:

In [ ]:
#欠損値を補完関数IterativeImputerのKNeighborsRegressorで補完
from sklearn.neighbors import KNeighborsRegressor
imp = IterativeImputer(estimator=KNeighborsRegressor(n_neighbors=15),max_iter=10,random_state=0)
df_train_fk = pd.DataFrame(imp.fit_transform(df_train))
df_train_fk.columns = df_train.columns
imp,score = run_model(df_train_fk,df_label)
print(score)
scorek = score

/usr/local/lib/python3.7/dist-packages/sklearn/impute/_iterative.py:701: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  ConvergenceWarning,


[Fold 1/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 264742	valid_1's rmse: 365463
[200]	training's rmse: 233795	valid_1's rmse: 357867
[300]	training's rmse: 213826	valid_1's rmse: 355399
[400]	training's rmse: 199351	valid_1's rmse: 353983
Early stopping, best iteration is:
[393]	training's rmse: 199988	valid_1's rmse: 353934
[Fold 2/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 274043	valid_1's rmse: 333207
[200]	training's rmse: 241843	valid_1's rmse: 324351
[300]	training's rmse: 222325	valid_1's rmse: 323127
[400]	training's rmse: 207544	valid_1's rmse: 321277
[500]	training's rmse: 196330	valid_1's rmse: 320064
Early stopping, best iteration is:
[468]	training's rmse: 199791	valid_1's rmse: 319964
[Fold 3/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 278602	valid_1's rmse: 307546
[200]	training's rmse: 246375	valid_1's rmse: 299324
[300]	training's rmse:

In [ ]:
#pandasで補完
dfp = df_train.interpolate(limit_direction='both')
df_train_fp=pd.DataFrame(dfp)
df_train_fp.columns=df_train.columns
imp,score=run_model(df_train_fp,df_label)
print(score)
scorep = score

[Fold 1/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 273020	valid_1's rmse: 370694
[200]	training's rmse: 241629	valid_1's rmse: 362518
[300]	training's rmse: 221099	valid_1's rmse: 359477
[400]	training's rmse: 204386	valid_1's rmse: 358412
[500]	training's rmse: 190158	valid_1's rmse: 358416
Early stopping, best iteration is:
[465]	training's rmse: 194785	valid_1's rmse: 358011
[Fold 2/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 280930	valid_1's rmse: 336735
[200]	training's rmse: 248748	valid_1's rmse: 330336
[300]	training's rmse: 226383	valid_1's rmse: 327104
[400]	training's rmse: 210338	valid_1's rmse: 326212
Early stopping, best iteration is:
[365]	training's rmse: 215324	valid_1's rmse: 326088
[Fold 3/5]
Training until validation scores don't improve for 50 rounds.
[100]	training's rmse: 284098	valid_1's rmse: 320110
[200]	training's rmse: 252894	valid_1's rmse: 313610
[300]	training's rmse:

4つを比較する

In [ ]:
print("結果")
print("何もしない", scored)
print("Baysian Ridge", scorei)
print("KNeighborsRegressor", scorek)
print("Pandasで補完", scorep)

結果
何もしない 300058.2489385326
Baysian Ridge 305965.3623000955
KNeighborsRegressor 308528.8593320058
Pandasで補完 317439.7811488135
